<a href="https://colab.research.google.com/github/ankit-rathi/Quantvesting_v2/blob/main/myStocks_Portfolio_Analysis.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install pyxirr
!pip install ta
!pip install yfinance==0.2.59

  Installing build dependencies ... -

 \

 done


  Getting requirements to build wheel ... -

 done


...

 done
  Created wheel for ta: filename=ta-0.11.0-py3-none-any.whl size=29482 sha256=58adf1471e39f6faecdda8c1caa5aac24fb3c0bc7413b67df71d94e9c9f435ae
  Stored in directory: /home/runner/.cache/pip/wheels/ec/b6/81/d781ff040ee9432384560f29596ee14ca79d70529d4228e50f
Successfully built ta


  Attempting uninstall: protobuf
    Found existing installation: protobuf 6.33.1
    Uninstalling protobuf-6.33.1:
      Successfully uninstalled protobuf-6.33.1


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 0/2 [protobuf]

  Attempting uninstall: yfinance
    Found existing installation: yfinance 0.2.66
    Uninstalling yfinance-0.2.66:
      Successfully uninstalled yfinance-0.2.66
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 0/2 [protobuf]

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2/2 [yfinance]


In [2]:
# import libraries

import numpy as np
import pandas as pd
import datetime
import warnings
warnings.filterwarnings('ignore')
import yfinance as yfin
import ta
import matplotlib.pyplot as plt
import requests

In [3]:
# notebook start time
import time
import datetime
import pytz

start_time = time.time()

# function to print date timestamp
def print_date_time():
  tz_NY = pytz.timezone('Asia/Kolkata')
  datetime_NY = datetime.datetime.now(tz_NY)
  print("Run date time (IST):", datetime_NY.strftime("%Y-%m-%d %H:%M:%S"))

In [4]:
# function to get booked and reserved amount
def get_amt():
  # fetch the JSON data from the URL
  url = "https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myPortfolioAmts.json"
  response = requests.get(url)
  pf_amts = response.json()  # parse the JSON data

  # extract values from the JSON
  py_booked_amt_dm = pf_amts["py_booked_amt_dm"]
  py_booked_amt_sv = pf_amts["py_booked_amt_sv"]
  cy_booked_amt_dm = pf_amts["cy_booked_amt_dm"]
  cy_booked_amt_sv = pf_amts["cy_booked_amt_sv"]
  reserve_amt_dm = pf_amts["reserve_amt_dm"]
  reserve_amt_sv = pf_amts["reserve_amt_sv"]

  # perform calculations
  py_booked_amt = py_booked_amt_dm + py_booked_amt_sv
  cy_booked_amt = cy_booked_amt_dm + cy_booked_amt_sv
  reserve_amt = reserve_amt_dm + reserve_amt_sv
  total_booked_amt = py_booked_amt + cy_booked_amt

  return total_booked_amt, reserve_amt, cy_booked_amt, py_booked_amt

gen_output = 0

In [5]:
# function to format the amount
def format_amt(number):
    abs_number = abs(number)

    if abs_number >= 1_00_00_000:  # Crores
        formatted_amt = f"{number / 1_00_00_000:.2f} C"
    elif abs_number >= 1_00_000:  # Lakhs
        formatted_amt = f"{number / 1_00_000:.2f} L"
    elif abs_number >= 1_000:  # Thousands
        formatted_amt = f"{number / 1_000:.2f} K"
    else:
        formatted_amt = f"{number:.2f}"

    return formatted_amt

# function to set start and end date
def get_start_end_date():
  start_date = (datetime.date.today() + datetime.timedelta(days=-365)).strftime('%Y-%m-%d')
  end_date = (datetime.date.today() + datetime.timedelta(days=1)).strftime('%Y-%m-%d')
  return start_date, end_date

# function to fetch my portfolio csv
def get_mypfs_df():
  mypfs_df = pd.read_csv('https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myPortfolioStocks.csv')
  return mypfs_df

# function to fetch my prospects csv
def get_mypps_df():
  mypps_df = pd.read_csv('https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myProspectsScrips.csv')
  return mypps_df

# function to fetch screener data
def get_myscreen_df():
  myscreen_df = pd.read_csv('https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myScreenerDB.csv')
  return myscreen_df

# function to fetch momentum data
def get_myinvmt_df():
  myinvmt_df = pd.read_csv('https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myInvestments.csv')
  return myinvmt_df

# function to get the stock ids
def get_stock_ids(df_pf):
  stock_n100 = df_pf['Symbol'].unique()

  exclude = ['CADILAHC','MMTC', 'MASFIN']

  stock_ids = df_pf[~df_pf['Symbol'].isin(exclude) ]['Symbol'].unique()

  #mypf = mypf[mypf.Forecast.notnull()]
  #stock_ids = mypf['Symbol'].unique()

  stock_ids.sort()
  return stock_ids

# get features from screener data
def get_screener_features():
  myscreen_df = get_myscreen_df()
  cols = ['Symbol', 'EPS', 'MedPE', 'ROCE%', 'ROE%', 'CapType']
  return myscreen_df[cols]

# get relative strength
def get_relative_strength(stock_list):

    # Dictionary to store stock tickers and their corresponding percentage price change
    stock_changes = {}

    # Loop through each stock and fetch its price data
    for stock in stock_list:
        try:
            # Download the stock data for the given date range
            stock_data = yfin.Ticker(stock + '.NS').history(period='1mo', interval='1d')[map(str.title, ['open', 'close', 'low', 'high', 'volume'])]

            # Calculate the percentage change for the stock
            if len(stock_data) > 0:
                start_price = stock_data['Close'].iloc[0]
                end_price = stock_data['Close'].iloc[-1]
                percent_change = round(((end_price - start_price) / start_price) * 100, 2)
                stock_changes[stock] = percent_change
            else:
                stock_changes[stock] = np.nan  # If no data is available, set to NaN

        except Exception as e:
            print(f"Error fetching data for {stock}: {e}")
            stock_changes[stock] = np.nan

    # Create a DataFrame with stock tickers and their percentage changes
    df = pd.DataFrame(list(stock_changes.items()), columns=['Symbol', 'Percent_Change'])

    # Drop any stocks with missing data (NaN values)
    df = df.dropna()

    # Calculate the percentile rank based on percentage change
    df['RSP'] = round(df['Percent_Change'].rank(pct=True) * 100, 2)
    cols = ['Symbol', 'RSP']
    df = df[cols]
    # Sort by percentile rank (optional)
    df = df.sort_values(by='RSP', ascending=False).reset_index(drop=True)

    return df


In [6]:
# function to get stock technicals
def stock_prec_dev(stock_symbol):
    #stock_symbol = 'ULTRACEMCO.NS'
    short_window = 20
    mid_window = 50
    long_window = 200
    moving_avg = 'SMA'

    start = datetime.datetime(*map(int, start_date.split('-')))
    end = datetime.datetime(*map(int, end_date.split('-')))

    stock_df = yfin.Ticker(stock_symbol + '.NS').history(period='max', interval='1d')[map(str.title, ['open', 'close', 'low', 'high', 'volume'])]

    stock_df['Max'] = round(max(stock_df['Close']),2)
    stock_df = stock_df[(stock_df.index <= end_date) & (stock_df.index >= start_date)]
    stock_df['200_SMA'] = round(stock_df['Close'].rolling(window = 200, min_periods = 1).mean(),0)
    stock_df['Dev%_200'] = round((stock_df['Close'] - stock_df['200_SMA'])*100/stock_df['200_SMA'],2)
    stock_df.dropna(axis = 0, inplace = True) # remove any null rows

    stock_df['20_SMA'] = stock_df['Close'].rolling(window=20).mean()
    stock_df['50_SMA'] = stock_df['Close'].rolling(window=50).mean()
    stock_df['Symbol'] = stock_symbol

    stock_df['Close'] = round(stock_df['Close'],2)
    stock_df['Min'] = round(min(stock_df['Close']),2)
    stock_df['RSI_14'] = ta.momentum.RSIIndicator(close=stock_df['Close'], window=14).rsi()
    stock_df['RSI_14'] = round(stock_df['RSI_14'],0)
    stock_df['Prev_Close'] = stock_df['Close'].shift(1)
    stock_df.drop(['Open', 'Low', 'High', 'Volume'], axis=1, inplace=True)
    stock_df = stock_df.tail(1)

    max_SMA = max(stock_df['20_SMA'].item(), stock_df['50_SMA'].item(), stock_df['200_SMA'].item())
    min_SMA = min(stock_df['20_SMA'].item(), stock_df['50_SMA'].item(), stock_df['200_SMA'].item())
    ABS_Spread = max_SMA - min_SMA
    stock_df['Spread%'] = round((ABS_Spread / stock_df['200_SMA'].item()) * 100,2)

    return stock_df

# function to compute stock attributes
def get_common_features(stock_ids, df_mypf):

  df_prec_dev = pd.DataFrame()
  df_tmp = get_screener_features()
  df_rs = get_relative_strength(stock_ids)

  for stock_id in stock_ids:
      tmp = stock_prec_dev(stock_id)
      tmp = tmp.reset_index()
      df_prec_dev = pd.concat([df_prec_dev, tmp], ignore_index = True)
  df_prec_dev = pd.merge(df_prec_dev, df_mypf, on= 'Symbol')
  df_prec_dev = pd.merge(df_prec_dev, df_tmp, on= 'Symbol', how='left')
  df_prec_dev = pd.merge(df_prec_dev, df_rs, on= 'Symbol', how='left')
  #print(df_prec_dev.columns)
  df_prec_dev['Curr_PE'] = round(df_prec_dev['Close']/df_prec_dev['EPS'],1)
  df_prec_dev['Dev%_PE'] = round((df_prec_dev['Curr_PE'] - df_prec_dev['MedPE'])*100/df_prec_dev['MedPE'],2)
  df_prec_dev['Conviction'] = df_prec_dev['Conviction'] + '-' + df_prec_dev['CapType']
  return df_prec_dev

# function to arrange stock features
def arrange_features(df_stocks, common_cols, diff_cols):
  df_stocks_common = df_stocks[common_cols].drop_duplicates()
  df_stocks_diff = df_stocks[diff_cols]
  df_stocks_diff['Investment'] = df_stocks_diff['AvgCost'] * df_stocks_diff['Shares']
  df_stocks_diff = df_stocks_diff.groupby(['Symbol'])[['Shares', 'Investment']].aggregate(['sum']).reset_index()
  df_stocks_diff.columns = ['Symbol', 'Shares', 'Investment']
  df_stocks_diff['AvgCost'] = round(df_stocks_diff['Investment']/df_stocks_diff['Shares'],2)
  df_stocks = pd.merge(df_stocks_diff, df_stocks_common, on='Symbol')
  return df_stocks

# plot fact distribution across dimension
def plot_pie_chart(df, dimension, fact):
  # grouping the data by category and calculating the sum of fact for each type
  grouped_data = df.groupby(dimension)[fact].sum()

  # sorting the grouped data in descending order
  grouped_data = grouped_data.sort_values(ascending=False)

  # creating a pie chart
  grouped_data.plot.pie(autopct='%1.1f%%', startangle=90, figsize=(6, 6))

  # adding a title and displaying the plot
  plt.title(f'{dimension} {fact} Distribution')
  plt.ylabel('')  # To hide the y-label
  plt.show()

In [7]:
# function to get portfolio features

def get_portfolio_features(df_common_features):

  print_date_time()
  print('-------------------')

  #df_common_features["Target"] = df_common_features["Target"].fillna(df_common_features["Max"])
  df_common_features['NTT'] = np.where(df_common_features['Strategy']== 'NTT', df_common_features["Target"], df_common_features['Max'])
  df_common_features['LTT'] = np.where(df_common_features['Strategy']== 'BTT', df_common_features["Target"], df_common_features['Max'])
  df_common_features['BOL'] = df_common_features['Min']

  tmp_df = df_common_features[df_common_features['Symbol'].isin(mypf_df[mypf_df['InPortfolio'] != 'NA'].Symbol.values)]
  print('qualified stocks: '+str(len(tmp_df['Symbol'].unique())))
  tmp_df1 = tmp_df[tmp_df['LatestQtr'] == 1]
  print('with latest results: '+str(len(tmp_df1['Symbol'].unique())))
  tmp_df1 = tmp_df1[tmp_df1['StarStock'] == 1]
  print('still star stocks: '+str(len(tmp_df1['Symbol'].unique())))
  tmp_df['Investment'] = tmp_df['AvgCost'] * tmp_df['Shares']
  tmp_df['Current'] = round(tmp_df['Close'] * tmp_df['Shares'],0)
  tmp_df['Previous'] = tmp_df['Prev_Close'] * tmp_df['Shares']
  tmp_df['EstimatedST'] = tmp_df['NTT'] * tmp_df['Shares']
  tmp_df['EstimatedLT'] = tmp_df['LTT'] * tmp_df['Shares']
  tmp_df['Current P/L'] = round((tmp_df['Current'] - tmp_df['Investment']),0)
  tmp_df['Today P/L%'] = round((tmp_df['Current'] - tmp_df['Previous'])*100/tmp_df['Previous'],2)
  tmp_df['Current P/L%'] = round((tmp_df['Current'] - tmp_df['Investment'])*100/tmp_df['Investment'],2)
  tmp_df['EstimatedST P/L%'] = round((tmp_df['EstimatedST'] - tmp_df['Investment'])*100/tmp_df['Investment'],2)
  tmp_df['EstimatedLT P/L%'] = round((tmp_df['EstimatedLT'] - tmp_df['Investment'])*100/tmp_df['Investment'],2)
  tmp_df['NTT%'] = round((tmp_df['NTT'] - tmp_df['Close'])*100/tmp_df['Close'],2)
  tmp_df['LTT%'] = round((tmp_df['LTT'] - tmp_df['Close'])*100/tmp_df['Close'],2)
  tmp_df['Gained%'] = round((tmp_df['Close'] - tmp_df['BOL'])*100/tmp_df['BOL'],2)
  investment = round(sum(tmp_df['AvgCost']*tmp_df['Shares']),0)
  current = round(sum(tmp_df['Close']*tmp_df['Shares']),0)
  tmp_df['InitAlloc%'] = round(tmp_df['Investment']*100/investment,2)
  tmp_df['CurrAlloc%'] = round(tmp_df['Current']*100/current,2)
  tmp_df['FTT'] = tmp_df['LTT']
  tmp_df.loc[tmp_df['Strategy'] == 'NTT', 'FTT'] = tmp_df['NTT']
  tmp_df['FTT%'] = tmp_df['LTT%']
  tmp_df.loc[tmp_df['Strategy'] == 'NTT', 'FTT%'] = tmp_df['NTT%']
  tmp_df['FTT Amt'] = round(tmp_df['FTT%'] * tmp_df['Current']/100,0)
  tmp_df['OTT%'] = round((tmp_df['FTT'] - tmp_df['AvgCost'])*100/tmp_df['AvgCost'],2)
  tmp_df['RRR Ind'] = round(tmp_df['Current P/L']/tmp_df['FTT Amt'],2)
  tmp_df['Risk Ind'] = round(tmp_df['Current P/L%']*tmp_df['CurrAlloc%'],0)

  return tmp_df

# function to print portfolio features
def print_portfolio_stats(df_portfolio_features, myinvmt_df):
  from pyxirr import xirr

  total_booked_amt, reserve_amt, cy_booked_amt, py_booked_amt = get_amt()

  dates = myinvmt_df['Date'].values
  dates = np.append(dates, datetime.date.today().strftime('%d-%b-%y'))
  investment = myinvmt_df['Investment'].values
  dates= pd.to_datetime(dates)

  current = round(sum(df_portfolio_features['Close']*df_portfolio_features['Shares']),0) + reserve_amt
  investment_xirr = np.append(investment, current)
  cagr = round(xirr(pd.DataFrame({"dates": dates, "amounts": investment_xirr}))*100,2)

  investment = -sum(investment)
  invested = round(sum(df_portfolio_features['AvgCost']*df_portfolio_features['Shares']),0) + reserve_amt
  previous = round(sum(df_portfolio_features['Prev_Close']*df_portfolio_features['Shares']),0) + reserve_amt
  cy_invested = investment + py_booked_amt

  today_pnl_amount = current-previous
  today_pnl_percentage = round((current-previous)*100/previous,2)

  curr_pnl_amount = current-invested
  curr_pnl_percentage = round((curr_pnl_amount)*100/(cy_invested),2)

  cy_pnl_amount = cy_booked_amt + curr_pnl_amount
  cy_pnl_percentage = round((cy_pnl_amount)*100/cy_invested,2)

  overall_pnl_amount = total_booked_amt + curr_pnl_amount
  overall_pnl_percentage = round((overall_pnl_amount)*100/investment,2)

  estimate_st = round(sum(df_portfolio_features['FTT']*df_portfolio_features['Shares']),0)  + reserve_amt
  est_st_pnl_amount = estimate_st-current
  est_st_pnl_percentage = round((est_st_pnl_amount)*100/current,2)

  estimate_lt = round(sum(df_portfolio_features['LTT']*df_portfolio_features['Shares']),0)  + reserve_amt
  est_lt_pnl_amount = estimate_lt-current
  est_lt_pnl_percentage = round((est_lt_pnl_amount)*100/current,2)

  total_profit = round(sum(df_portfolio_features[df_portfolio_features['Current P/L%'] > 0]['Current']) - sum(df_portfolio_features[df_portfolio_features['Current P/L%'] > 0]['Investment']),0)
  total_loss = round(sum(df_portfolio_features[df_portfolio_features['Current P/L%'] < 0]['Current']) - sum(df_portfolio_features[df_portfolio_features['Current P/L%'] < 0]['Investment']),0)

  cy_booked_percentage = round((cy_booked_amt)*100/current,2)
  py_booked_percentage = round((py_booked_amt)*100/investment,2)
  total_booked_percentage = round((total_booked_amt)*100/investment,2)

  print('-------------------')
  print('Initial Investment: ', format_amt(investment))
  print('CY Investment: ', format_amt(cy_invested))
  print('Reserve: ', format_amt(reserve_amt))
  print('Current: ',  format_amt(current))
  print('-------------------')
  print('Today PnL: '+ '{} ({}%)'.format(format_amt(today_pnl_amount), today_pnl_percentage))
  print('Current PnL: '+ '{} ({}%)'.format(format_amt(curr_pnl_amount), curr_pnl_percentage))
  print('CY Booked + Current PnL: '+ '{} ({}%)'.format(format_amt(cy_pnl_amount), cy_pnl_percentage))
  print('-------------------')
  print('Total profit: ', format_amt(total_profit))
  print('Total loss: ', format_amt(total_loss))
  print('-------------------')
  print('Total Booked + Current PnL: '+ '{} ({}%)'.format(format_amt(overall_pnl_amount), overall_pnl_percentage))
  print('Total Booked PnL: '+ '{} ({}%)'.format(format_amt(total_booked_amt), total_booked_percentage))
  print('Curr Year Booked PnL: '+ '{} ({}%)'.format(format_amt(cy_booked_amt), cy_booked_percentage))
  print('Prev Year Booked PnL: '+ '{} ({}%)'.format(format_amt(py_booked_amt), py_booked_percentage))
  print('===================')
  print('Est FTT: ',  format_amt(estimate_st))
  print('Est FTT PnL: '+ '{} ({}%)'.format(format_amt(est_st_pnl_amount), est_st_pnl_percentage))

  print('===================')
  print('Deployed: ', format_amt(investment))

  print('Current: ', format_amt(current))

  print('CAGR/XIRR %: '+'{}%'.format(cagr))

In [8]:
# get start and end date
start_date, end_date = get_start_end_date()

# get portfolio and prospects data
mypfs_df = get_mypfs_df()
mypps_df = get_mypps_df()
myinvmt_df = get_myinvmt_df()

# merge above datasets
mypf_df = pd.merge(mypfs_df, mypps_df, on="Symbol")

# seggregate dm and sv portfolio
dm_pf = mypf_df[mypf_df['InPortfolio'] == 'DM']
sv_pf = mypf_df[mypf_df['InPortfolio'] == 'SV']

# build portfolio stock dataframe
dm_stocks = get_stock_ids(dm_pf)
sv_stocks = get_stock_ids(sv_pf)
df_stocks = pd.concat([dm_pf,sv_pf], ignore_index = True)

# arrange common and diff stock features
common_cols = ['Symbol', 'Target', 'Criteria', 'Strategy', 'CumlRnk', 'LatestQtr', 'StarStock', 'Conviction', 'Category', 'InFolio', 'XIRR', 'MBQ']
diff_cols = ['Symbol', 'AvgCost', 'Shares']
df_stocks = arrange_features(df_stocks, common_cols, diff_cols)

# get common features
stock_ids = df_stocks['Symbol'].values
df_common_features = get_common_features(stock_ids, df_stocks)

df_common_features.reset_index(drop=True, inplace=True)
df_common_features.drop(['Date'], axis=1, inplace=True)
# get and print portfolio features
df_portfolio_features = get_portfolio_features(df_common_features)

print_portfolio_stats(df_portfolio_features, myinvmt_df)

df = df_portfolio_features
#plot_pie_chart(df, 'CapType', 'Current')
list_ox40n = list(df[df['MBQ'].str.contains('OX40N', na=False)]['Symbol'].values)

Run date time (IST): 2025-11-17 10:42:52
-------------------
qualified stocks: 87
with latest results: 26
still star stocks: 16
-------------------
Initial Investment:  1.26 C
CY Investment:  1.53 C
Reserve:  3.00 K
Current:  1.46 C
-------------------
Today PnL: 31.68 K (0.22%)
Current PnL: -19.90 L (-13.04%)
CY Booked + Current PnL: -6.38 L (-4.18%)
-------------------
Total profit:  2.05 L
Total loss:  -21.94 L
-------------------
Total Booked + Current PnL: 20.60 L (16.4%)
Total Booked PnL: 40.50 L (32.24%)
Curr Year Booked PnL: 13.52 L (9.29%)
Prev Year Booked PnL: 26.98 L (21.48%)
Est FTT:  2.36 C
Est FTT PnL: 90.78 L (62.38%)
Deployed:  1.26 C
Current:  1.46 C
CAGR/XIRR %: 8.5%


In [9]:
# top 5 near their targets
cols = ['Symbol', 'FTT', 'Dev%_PE', 'RSI_14','Conviction', 'Spread%', 'Current', 'Current P/L', 'FTT Amt', 'Today P/L%', 'Current P/L%', 'FTT%', 'OTT%', 'CumlRnk', 'RRR Ind', 'CurrAlloc%', 'Gained%', 'Criteria', 'Strategy', 'Category']
df_tmp = df_portfolio_features[~df_portfolio_features['Symbol'].isin(['ENRIN','BLUSPRING','DIGITIDE'])]
df_tmp = df_tmp.sort_values(by = 'FTT Amt', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
78,TTKPRESTIG,770.00,101.92,52.0,M-SC,3.43,87017.0,-13760.0,13853.0,0.11,-13.65,15.92,0.09,253.0,-0.99,0.60,14.32,OX40N,NTT,DURABLES
39,INDIGOPNTS,1408.00,142.19,79.0,M-SC,0.83,158844.0,-15715.0,15741.0,-1.37,-9.00,9.91,0.02,234.0,-1.00,1.09,38.17,OX40N,NTT,PAINTS
2,ABBOTINDIA,35195.00,-15.95,42.0,X-MC,3.60,87810.0,-2868.0,17773.0,-0.24,-3.16,20.24,16.44,63.0,-0.16,0.60,16.37,X40,ATH,PHARMA
5,ANGELONE,3033.00,62.97,77.0,X-SC,7.37,228944.0,37938.0,19758.0,1.70,19.86,8.63,30.21,97.0,1.92,1.57,44.89,X40N,NTT,FINANCE
35,ICICIGI,2252.93,-15.31,56.0,X-MC,7.49,188195.0,13875.0,21322.0,0.92,7.96,11.33,20.19,68.0,0.65,1.29,24.28,X40,ATH,INSURANCE


In [10]:
# top 5 today
df_tmp = df_portfolio_features.sort_values(by = 'Today P/L%', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
70,SURYODAY,240.0,66.67,57.0,H-SC,13.77,153714.0,-25357.0,91091.0,9.58,-14.16,59.26,36.71,167.0,-0.28,1.06,52.51,XR,NTT,BANKS
66,SIEMENS,4671.5,-2.24,59.0,H-LC,1.55,159375.0,-26720.0,74205.0,3.35,-14.36,46.56,25.51,49.0,-0.36,1.10,17.45,AR,ATH,ELECTRICAL
83,VOLTAS,1530.0,-2.18,53.0,X-MC,3.76,207840.0,16098.0,21657.0,2.57,8.40,10.42,19.69,78.0,0.74,1.43,15.76,XY25,NTT,AC
56,RAJESHEXPO,518.0,1766.16,66.0,L-SC,2.81,50920.0,-86257.0,86350.0,2.57,-62.88,169.58,0.07,268.0,-1.00,0.35,26.64,OX40N,NTT,JEWELLERY
62,SAMMAANCAP,326.0,19.75,57.0,M-SC,36.94,161919.0,12699.0,131478.0,2.14,8.51,81.20,96.62,245.0,0.10,1.11,75.23,XY25,NTT,FINANCE


In [11]:
# bottom 5 today
df_tmp = df_portfolio_features.sort_values(by = 'Today P/L%', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
76,TMPV,1065.0,-20.63,26.0,X-LC,0.30,245494.0,-30916.0,452077.0,-4.19,-11.18,184.15,152.37,3.0,-0.07,1.69,0.00,XY24,NTT,AUTO
19,COFFEEDAY,80.0,-51.71,32.0,L-SC,21.17,71040.0,-42509.0,82563.0,-3.65,-37.44,116.22,35.27,270.0,-0.51,0.49,73.87,XR,NTT,HOTELS
24,EASEMYTRIP,26.4,-21.71,33.0,M-SC,20.25,75527.0,-101288.0,182088.0,-2.64,-57.28,241.09,45.70,205.0,-0.56,0.52,0.00,XY24,NTT,TRAVEL
14,BSOFT,831.7,-11.09,58.0,H-SC,8.62,97614.0,-43039.0,114472.0,-1.52,-30.60,117.27,50.79,151.0,-0.38,0.67,13.65,XR,ATH,IT
39,INDIGOPNTS,1408.0,142.19,79.0,M-SC,0.83,158844.0,-15715.0,15741.0,-1.37,-9.00,9.91,0.02,234.0,-1.00,1.09,38.17,OX40N,NTT,PAINTS


In [12]:
# top 5 to exit based on CumlRnk
df_tmp = df_portfolio_features[~df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])]
df_tmp = df_tmp[(df_tmp['Current P/L%'] > -1) & (df_tmp['Current P/L%'] < 1)].sort_values(by = 'CumlRnk', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
25,FINCABLES,1641.55,-18.52,49.0,M-SC,10.21,147982.0,338.0,158992.0,1.06,0.23,107.44,107.91,216.0,0.00,1.02,3.24,OX40N,ATH,CABLES
86,ZYDUSLIFE,1286.17,-14.97,36.0,H-MC,5.60,207574.0,-2000.0,76678.0,-0.55,-0.95,36.94,35.63,119.0,-0.03,1.43,15.56,AR,ATH,PHARMA
85,WIPRO,420.00,-14.25,50.0,M-LC,5.60,150753.0,-192.0,109643.0,-0.50,-0.13,72.73,72.51,99.0,-0.00,1.04,5.89,XR,NTT,IT


In [13]:
# OX40N stocks to exit
df_tmp = df_portfolio_features[(df_portfolio_features['Symbol'].isin(list_ox40n))].sort_values(by = 'Current P/L%', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
25,FINCABLES,1641.55,-18.52,49.0,M-SC,10.21,147982.0,338.0,158992.0,1.06,0.23,107.44,107.91,216.0,0.00,1.02,3.24,OX40N,ATH,CABLES
39,INDIGOPNTS,1408.00,142.19,79.0,M-SC,0.83,158844.0,-15715.0,15741.0,-1.37,-9.00,9.91,0.02,234.0,-1.00,1.09,38.17,OX40N,NTT,PAINTS
78,TTKPRESTIG,770.00,101.92,52.0,M-SC,3.43,87017.0,-13760.0,13853.0,0.11,-13.65,15.92,0.09,253.0,-0.99,0.60,14.32,OX40N,NTT,DURABLES
47,KANSAINER,340.00,-66.17,48.0,H-SC,2.35,222417.0,-47250.0,83584.0,-0.67,-17.52,37.58,13.47,158.0,-0.57,1.53,13.09,XY24,NTT,PAINTS
67,SIS,528.00,2017.62,46.0,H-SC,2.37,85450.0,-25582.0,49185.0,-0.33,-23.04,57.56,21.26,166.0,-0.52,0.59,15.45,OX40N,NTT,MISC


In [14]:
# top 5 to book for rotation
df_tmp = df_portfolio_features[(df_portfolio_features['Current P/L%'] > 20) ].sort_values(by = 'CumlRnk', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category


In [15]:
# top 5 to monitor for exit
df_tmp = df_portfolio_features[~df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])]
df_tmp = df_tmp[(df_tmp['Current P/L%'] > 0) ].sort_values(by = 'Dev%_PE', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
69,STARHEALTH,761.00,39.16,73.0,H-SC,15.58,286470.0,5347.0,127508.0,0.79,1.90,44.51,47.26,171.0,0.04,1.97,53.48,XY24,NTT,INSURANCE
62,SAMMAANCAP,326.00,19.75,57.0,M-SC,36.94,161919.0,12699.0,131478.0,2.14,8.51,81.20,96.62,245.0,0.10,1.11,75.23,XY25,NTT,FINANCE
25,FINCABLES,1641.55,-18.52,49.0,M-SC,10.21,147982.0,338.0,158992.0,1.06,0.23,107.44,107.91,216.0,0.00,1.02,3.24,OX40N,ATH,CABLES
38,INDIAMART,4810.62,-47.77,51.0,H-SC,4.42,130094.0,6758.0,124864.0,0.13,5.48,95.98,106.72,139.0,0.05,0.89,30.14,AR,ATH,MISC


In [16]:
# top 5 to book for rotation from weak conviction
df_tmp = df_portfolio_features[~df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])]
df_tmp = df_tmp[~df_tmp['Criteria'].isin(['XY25','XY24','X40', 'X40N'])].sort_values(by = 'RRR Ind', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
38,INDIAMART,4810.62,-47.77,51.0,H-SC,4.42,130094.0,6758.0,124864.0,0.13,5.48,95.98,106.72,139.0,0.05,0.89,30.14,AR,ATH,MISC
25,FINCABLES,1641.55,-18.52,49.0,M-SC,10.21,147982.0,338.0,158992.0,1.06,0.23,107.44,107.91,216.0,0.00,1.02,3.24,OX40N,ATH,CABLES
85,WIPRO,420.00,-14.25,50.0,M-LC,5.60,150753.0,-192.0,109643.0,-0.50,-0.13,72.73,72.51,99.0,-0.00,1.04,5.89,XR,NTT,IT
84,WHIRLPOOL,2270.00,-42.22,38.0,M-SC,8.85,113289.0,-1509.0,95548.0,-1.09,-1.31,84.34,81.92,236.0,-0.02,0.78,34.12,XR,NTT,DURABLES
86,ZYDUSLIFE,1286.17,-14.97,36.0,H-MC,5.60,207574.0,-2000.0,76678.0,-0.55,-0.95,36.94,35.63,119.0,-0.03,1.43,15.56,AR,ATH,PHARMA


In [17]:
# top 5 to accumulate based on lowest RSI_14
df_tmp = df_portfolio_features[df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])].sort_values(by = 'RSI_14', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
12,BATAINDIA,2096.0,11.05,25.0,X-SC,9.76,84643.0,-44027.0,89324.0,-0.71,-34.22,105.53,35.20,92.0,-0.49,0.58,0.00,X40,NTT,FOOTWEAR
76,TMPV,1065.0,-20.63,26.0,X-LC,0.30,245494.0,-30916.0,452077.0,-4.19,-11.18,184.15,152.37,3.0,-0.07,1.69,0.00,XY24,NTT,AUTO
55,QUESS,986.0,-53.17,30.0,X-SC,37.25,46682.0,-18324.0,169250.0,0.62,-28.19,362.56,232.18,83.0,-0.11,0.32,0.62,XY24,NTT,MISC
46,JSWINFRA,342.0,-19.06,31.0,X-MC,5.67,189405.0,-10942.0,41442.0,-0.11,-5.46,21.88,15.23,66.0,-0.26,1.30,25.03,X40N,NTT,REALTY
27,GLAXO,3466.2,-19.47,31.0,X-MC,6.39,199752.0,468.0,77544.0,0.71,0.23,38.82,39.15,60.0,0.01,1.37,29.30,X40,ATH,PHARMA


In [18]:
# top 5 to accumulate based on Spread and CumlRnk
df_tmp = df_portfolio_features[df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])]
df_tmp = df_tmp[(df_tmp['CumlRnk'] < 100)].sort_values(by = 'Spread%', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
76,TMPV,1065.00,-20.63,26.0,X-LC,0.30,245494.0,-30916.0,452077.0,-4.19,-11.18,184.15,152.37,3.0,-0.07,1.69,0.00,XY24,NTT,AUTO
43,ITC,452.00,-40.74,47.0,X-LC,0.91,199145.0,-993.0,21886.0,-0.22,-0.50,10.99,10.44,5.0,-0.05,1.37,5.04,X40,NTT,FMCG
34,HONAUT,58357.33,-25.23,35.0,X-SC,1.00,106365.0,-21573.0,68712.0,-0.21,-16.86,64.60,36.84,90.0,-0.31,0.73,8.89,X40N,ATH,ELECTRICAL
36,ICICIPRULI,790.00,-20.46,64.0,X-MC,1.17,183726.0,6988.0,47750.0,-0.38,3.95,25.99,30.97,75.0,0.15,1.26,17.01,X40,ATH,INSURANCE
66,SIEMENS,4671.50,-2.24,59.0,H-LC,1.55,159375.0,-26720.0,74205.0,3.35,-14.36,46.56,25.51,49.0,-0.36,1.10,17.45,AR,ATH,ELECTRICAL


In [19]:
# top 5 to accumulate based on lowest Gained%
df_tmp = df_portfolio_features[df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])].sort_values(by = 'Gained%', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
12,BATAINDIA,2096.00,11.05,25.0,X-SC,9.76,84643.0,-44027.0,89324.0,-0.71,-34.22,105.53,35.20,92.0,-0.49,0.58,0.00,X40,NTT,FOOTWEAR
76,TMPV,1065.00,-20.63,26.0,X-LC,0.30,245494.0,-30916.0,452077.0,-4.19,-11.18,184.15,152.37,3.0,-0.07,1.69,0.00,XY24,NTT,AUTO
55,QUESS,986.00,-53.17,30.0,X-SC,37.25,46682.0,-18324.0,169250.0,0.62,-28.19,362.56,232.18,83.0,-0.11,0.32,0.62,XY24,NTT,MISC
53,PAGEIND,51769.93,-27.96,41.0,X-MC,7.72,160060.0,-3680.0,47026.0,0.63,-2.25,29.38,26.47,55.0,-0.08,1.10,1.41,X40,ATH,APPARELS
10,BAJAJHFL,181.50,-18.58,36.0,X-MC,6.62,176120.0,-24386.0,121540.0,-0.14,-12.16,69.01,48.45,64.0,-0.20,1.21,1.56,X40N,ATH,FINANCE


In [20]:
# top 5 to accumulate based on lowest CurrAlloc%
df_tmp = df_portfolio_features[df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])].sort_values(by = 'CurrAlloc%', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
55,QUESS,986.00,-53.17,30.0,X-SC,37.25,46682.0,-18324.0,169250.0,0.62,-28.19,362.56,232.18,83.0,-0.11,0.32,0.62,XY24,NTT,MISC
59,RELAXO,1176.00,-40.86,42.0,X-SC,3.22,80643.0,-64517.0,140448.0,0.66,-44.45,174.16,52.31,91.0,-0.46,0.55,7.94,X40N,NTT,FOOTWEAR
12,BATAINDIA,2096.00,11.05,25.0,X-SC,9.76,84643.0,-44027.0,89324.0,-0.71,-34.22,105.53,35.20,92.0,-0.49,0.58,0.00,X40,NTT,FOOTWEAR
2,ABBOTINDIA,35195.00,-15.95,42.0,X-MC,3.60,87810.0,-2868.0,17773.0,-0.24,-3.16,20.24,16.44,63.0,-0.16,0.60,16.37,X40,ATH,PHARMA
34,HONAUT,58357.33,-25.23,35.0,X-SC,1.00,106365.0,-21573.0,68712.0,-0.21,-16.86,64.60,36.84,90.0,-0.31,0.73,8.89,X40N,ATH,ELECTRICAL


In [21]:
# top 5 to accumulate based on CumlRnk
df_tmp = df_portfolio_features.sort_values(by = 'CumlRnk', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
74,TCS,4389.97,-26.86,57.0,X-LC,8.49,290592.0,-55364.0,122078.0,-0.47,-16.00,42.01,19.28,1.0,-0.45,2.00,7.43,X40,ATH,IT
41,INFY,2275.00,-20.67,51.0,X-LC,4.01,318797.0,5909.0,165774.0,-0.41,1.89,52.00,54.87,2.0,0.04,2.19,10.33,X40,BTT,IT
76,TMPV,1065.00,-20.63,26.0,X-LC,0.30,245494.0,-30916.0,452077.0,-4.19,-11.18,184.15,152.37,3.0,-0.07,1.69,0.00,XY24,NTT,AUTO
82,VBL,671.64,-17.80,49.0,X-LC,5.98,294628.0,-21214.0,133879.0,0.52,-6.72,45.44,35.67,4.0,-0.16,2.03,6.21,X40N,ATH,FMCG
43,ITC,452.00,-40.74,47.0,X-LC,0.91,199145.0,-993.0,21886.0,-0.22,-0.50,10.99,10.44,5.0,-0.05,1.37,5.04,X40,NTT,FMCG


In [22]:
# top 5 for average up
df_tmp = df_portfolio_features[(df_portfolio_features['Dev%_200'] > 5)].sort_values(by = 'CurrAlloc%', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
19,COFFEEDAY,80.00,-51.71,32.0,L-SC,21.17,71040.0,-42509.0,82563.0,-3.65,-37.44,116.22,35.27,270.0,-0.51,0.49,73.87,XR,NTT,HOTELS
7,ASIANTILES,137.00,7616.67,65.0,L-SC,17.13,84692.0,-9118.0,85734.0,-0.12,-9.72,101.23,81.67,271.0,-0.11,0.58,64.01,XR,NTT,CERAMICS
51,MASFIN,398.61,-17.80,54.0,H-SC,7.49,93360.0,-4620.0,26225.0,0.63,-4.72,28.09,22.05,164.0,-0.18,0.64,35.71,XR,ATH,FINANCE
80,VAIBHAVGBL,521.00,73.55,62.0,H-SC,9.47,150306.0,-32469.0,143542.0,0.53,-17.76,95.50,60.77,159.0,-0.23,1.03,35.36,XR,NTT,JEWELLERY
70,SURYODAY,240.00,66.67,57.0,H-SC,13.77,153714.0,-25357.0,91091.0,9.58,-14.16,59.26,36.71,167.0,-0.28,1.06,52.51,XR,NTT,BANKS


In [23]:
# top 5 RSP
df_tmp = df_portfolio_features.sort_values(by = 'RSP', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
39,INDIGOPNTS,1408.00,142.19,79.0,M-SC,0.83,158844.0,-15715.0,15741.0,-1.37,-9.00,9.91,0.02,234.0,-1.00,1.09,38.17,OX40N,NTT,PAINTS
80,VAIBHAVGBL,521.00,73.55,62.0,H-SC,9.47,150306.0,-32469.0,143542.0,0.53,-17.76,95.50,60.77,159.0,-0.23,1.03,35.36,XR,NTT,JEWELLERY
6,ASIANPAINT,3465.66,0.35,85.0,X-LC,9.52,255068.0,3300.0,49917.0,-0.27,1.31,19.57,21.13,31.0,0.07,1.75,37.20,X40,ATH,PAINTS
40,INDUSINDBK,1800.00,-699.18,70.0,L-MC,6.17,48521.0,-28686.0,54077.0,0.35,-37.15,111.45,32.89,259.0,-0.53,0.33,33.62,XR,NTT,BANKS
14,BSOFT,831.70,-11.09,58.0,H-SC,8.62,97614.0,-43039.0,114472.0,-1.52,-30.60,117.27,50.79,151.0,-0.38,0.67,13.65,XR,ATH,IT


In [24]:
# Top N allocation
df_tmp = df_portfolio_features.sort_values(by = 'CurrAlloc%', ascending=False)
top_n_values = [10, 25, 50]

sum_df = pd.DataFrame({
    'Top_N': top_n_values,
    'Sum_Alloc%': [df_tmp['CurrAlloc%'].head(n).sum() for n in top_n_values]
})

sum_df

,Top_N,Sum_Alloc%
0,10,19.61
1,25,44.46
2,50,75.73


In [25]:
# market-cap-wise allocation
df_tmp = df_portfolio_features[cols]
df_tmp.groupby(df_tmp['Conviction'].str[-2:])['CurrAlloc%'].sum().sort_values(ascending=False)

Conviction
SC    47.64
MC    27.74
LC    24.63
Name: CurrAlloc%, dtype: float64

In [26]:
# criteria-wise allocation
df_portfolio_features.groupby('Criteria')['CurrAlloc%'].sum().sort_values(ascending=False)

Criteria
XY24     27.39
X40      20.45
X40N     12.27
XY25     10.11
XR        9.61
AR        9.03
OX40N     7.52
X200      1.80
SR        1.01
MH        0.82
Name: CurrAlloc%, dtype: float64

In [27]:
# conviction-wise allocation
df_portfolio_features.groupby('Conviction')['CurrAlloc%'].sum().sort_values(ascending=False)

Conviction
H-SC    24.33
X-MC    21.37
X-LC    19.70
M-SC    13.31
X-SC     8.58
H-MC     4.67
H-LC     2.90
L-SC     1.42
M-MC     1.37
M-LC     1.04
L-LC     0.99
L-MC     0.33
Name: CurrAlloc%, dtype: float64

In [28]:
# sector-wise stats
df_tmp = df_portfolio_features.groupby('Category')[['CurrAlloc%', 'Current', 'Current P/L', 'FTT Amt']].sum().sort_values(by=['Current', 'Current P/L'], ascending=False)
df_tmp['Current P/L%'] = round(df_tmp['Current P/L'] * 100 / df_tmp['Current'], 2)
df_tmp['FTT%'] = round(df_tmp['FTT Amt'] * 100 / df_tmp['Current'], 2)
cols = ['CurrAlloc%', 'Current P/L%', 'FTT%']
df_tmp[cols].sort_values(by=['CurrAlloc%'], ascending=False)

,CurrAlloc%,Current P/L%,FTT%
Category,,,
FMCG,16.64,-4.05,37.70
IT,12.44,-16.26,78.41
FINANCE,11.97,-8.49,57.67
MISC,7.53,-18.43,78.17
PAINTS,5.98,-5.98,21.78
ELECTRICAL,5.60,-11.02,50.37
INSURANCE,4.52,3.98,29.86
PHARMA,3.40,-0.89,34.74
AUTO,3.37,-17.27,129.70


In [29]:
# money to be made criteria-wise
df_portfolio_features.groupby('Criteria')['FTT Amt'].agg(['sum', 'count']).sort_values(by='sum', ascending=False)

,sum,count
Criteria,,
XY24,3322843.0,20
AR,1259927.0,10
XR,1256737.0,13
X40,992083.0,14
OX40N,697108.0,10
X40N,682408.0,9
XY25,465517.0,7
SR,273055.0,2
MH,65319.0,1


In [30]:
# money to be made conviction-wise
df_portfolio_features.groupby('Conviction')['FTT Amt'].agg(['sum', 'count']).sort_values(by='sum', ascending=False)

,sum,count
Conviction,,
H-SC,3463032.0,25
M-SC,1416312.0,16
X-LC,1203687.0,11
X-MC,1131156.0,15
X-SC,826044.0,8
H-MC,399772.0,3
L-SC,254647.0,3
H-LC,136698.0,2
M-LC,109643.0,1


In [31]:
# money to be made criteria and conviction-wise
df_portfolio_features.groupby(['Conviction', 'Criteria'])['FTT Amt'].agg(['sum', 'count']).sort_values(by='sum', ascending=False)

sum  count
Conviction Criteria                  
H-SC       XY24      1164236.0      6
           AR         841990.0      5
           XR         781403.0      7
M-SC       XY24       742901.0      6
X-LC       XY24       514641.0      2
           X40        412086.0      5
X-MC       X40        410256.0      7
X-SC       XY24       375377.0      2
X-MC       XY24       344761.0      3
H-SC       OX40N      337029.0      4
X-SC       X40N       280926.0      4
M-SC       OX40N      273729.0      5
H-SC       SR         273055.0      2
H-MC       AR         218845.0      2
X-LC       X40N       201404.0      2
X-MC       X40N       200078.0      3
H-MC       XY24       180927.0      1
X-MC       XY25       176061.0      2
X-SC       X40        169741.0      2
L-SC       XR         168297.0      2
M-SC       XR         143317.0      2
           XY25       131478.0      1
           AR         124887.0      2
M-LC       XR         109643.0      1
L-SC       OX40N       86350.0      1
X-LC       XY25        75556.0      2
H-LC       AR          74205.0      1
H-SC       MH          65319.0      1
H-LC       X200        62493.0      1
L-MC       XR          54077.0      1
M-MC       XY25        46372.0      1
L-LC       XY25        36050.0      1

In [32]:
# notebook execution time

end_time = time.time()
execution_time = round(end_time - start_time, 0)
print(f"Notebook execution time: {execution_time} seconds")

Notebook execution time: 74.0 seconds
